# ButtPlug Wars

## Setup

In [ ]:
import * as hardhat from 'hardhat'
import '@nomiclabs/hardhat-ethers'
import { ethers } from 'hardhat'
import { smock } from '@defi-wonderland/smock'

import * as bn from './utils/bn'
import * as evm from './utils/evm'
import * as wallet from './utils/wallet'
import * as contracts from './utils/contracts'

import {getMainnetSdk} from '@dethcrypto/eth-sdk-client';

In [ ]:
const FIVE_OUT_OF_NINE = '0xB543F9043b387cE5B3d1F0d916E42D8eA2eBA2E0';
const WETH_9 = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2';
const KEEP3R = '0xeb02addCfD8B773A5FFA6B9d1FE99c566f8c44CC';
const KP3R_LP = '0x3f6740b5898c5D3650ec6eAce9a649Ac791e44D7';
const SUDOSWAP_FACTORY = '0xb16c1342E617A5B6E4b631EB114483FDB289c0A4';
const SUDOSWAP_XYK_CURVE = '0x7942E264e21C5e6CbBA45fe50785a15D3BEb1DA0';

const FIVEOUTOFNINE_WHALE = '0xC5233C3b46C83ADEE1039D340094173f0f7c1EcF'
const KEEPER_ADDRESS = '0x9429cd74a3984396f3117d51cde46ea8e0e21487'

In [ ]:
// FORK
let blockNumber = 16000000

await evm.reset({
    jsonRpcUrl: process.env['MAINNET_RPC'],
    blockNumber
})

const provider = ethers.provider

In [ ]:
// Players
const signer = await ethers.getSigner()
const whale = await wallet.impersonate(FIVEOUTOFNINE_WHALE)
const keeper = await wallet.impersonate(KEEPER_ADDRESS)

In [ ]:
// Deployment
const args = [FIVE_OUT_OF_NINE, WETH_9, KEEP3R, KP3R_LP, SUDOSWAP_FACTORY, SUDOSWAP_XYK_CURVE]
const game = await contracts.deploy(signer, './out/ButtPlugWars.sol/ButtPlugWars.json', args)

In [ ]:
const { chess, sudoPool } = getMainnetSdk(signer)

In [ ]:
// JSON.parse(atob((await chess._tokenURI(100)).substring(29,1e6)))

## Playground

### Event start

In [ ]:
await evm.advanceTimeAndBlock(86400*10)
const tx = await game.startEvent();

### Player Badge minting

In [ ]:
// pre-genesis tokens
await chess.connect(whale).transferFrom(whale._address, signer.address, 133)
await chess.connect(whale).transferFrom(whale._address, signer.address, 134)
await chess.connect(whale).transferFrom(whale._address, signer.address, 135)
await chess.connect(whale).transferFrom(whale._address, signer.address, 136)
await chess.connect(whale).transferFrom(whale._address, signer.address, 137)
await chess.connect(whale).transferFrom(whale._address, signer.address, 138)
await chess.setApprovalForAll(game.address, true)

await game.mintPlayerBadge(133, 0, {value: bn.toUnit(1)})
await game.mintPlayerBadge(134, 0, {value: bn.toUnit(0.5)})
await game.mintPlayerBadge(135, 0, {value: bn.toUnit(0.25)})
await game.mintPlayerBadge(136, 0, {value: bn.toUnit(0.10)})
await game.mintPlayerBadge(137, 0, {value: bn.toUnit(0.10)})
const tx = await game.mintPlayerBadge(138, 1, {value: bn.toUnit(1)})

In [ ]:
await game['getBadgeId(uint256)'](5)

In [ ]:
await game['getBadgeId(uint256)'](6)

### ButtPlug Badge minting

In [ ]:
const buttPlug = await contracts.deploy(signer,'./out/Common.sol/ButtPlugForTest.json',[])

const tx = await game.mintButtPlugBadge(buttPlug.address)

### Game start

In [ ]:
await evm.advanceTimeAndBlock(14 * 86400)
const tx = await game.pushLiquidity()

### Voting

In [ ]:
const tx = await game['voteButtPlug(address,uint256)'](buttPlug.address, 1);
// const tx = await game['voteButtPlug(address,uint256[])'](buttPlug.address, [1, 2]);

await contracts.logTx(tx)

## Gameplay

In [ ]:
const officialSudoPool = sudoPool.attach(await game.SUDOSWAP_POOL())

In [ ]:
await chess.board()

### E2E environment

In [ ]:
await chess.connect(whale).transferFrom(whale._address, keeper._address, 175)
'keepers need a 5/9'

In [ ]:
await buttPlug.setDepth(7)
await evm.advanceTimeAndBlock(5*86400)
const tx = await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

In [ ]:
const tx = await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

### Post Genesis badges

In [ ]:
const receipt = await tx.wait()
const postGenesisToken = receipt.events[3].topics[3]

await game.mintPlayerBadge(postGenesisToken, 1, {value: bn.toUnit(0.1)})

In [ ]:
await game['getBadgeId(uint256)'](7)

### Mocked environment

In [ ]:
const iButtPlug = await contracts.getContractAbi('./out/IGame.sol/IButtPlug.json')
const fakeButtPlug = await smock.fake(iButtPlug)

#### Mock vs FiveOutOfNine

In [ ]:
const tx = await game['voteButtPlug(address,uint256[])'](fakeButtPlug.address, [1, 2, 3, 4, 5]);

In [ ]:
// Game #6: The Royal Entrance

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((10 << 6) | 25)
await game.connect(keeper).executeMove()

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((13 << 6) | 30)
await game.connect(keeper).executeMove()

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((20 << 6) | 28)
await game.connect(keeper).executeMove()

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((12 << 6) | 28)
await game.connect(keeper).executeMove()

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((28 << 6) | 42)
await game.connect(keeper).executeMove()

#### Fake Wars

In [ ]:
const chessAbi = require('./eth-sdk/abis/mainnet/chess.json')
const keep3rAbi = await contracts.getContractAbi('./out/IKeep3r.sol/IKeep3r.json')
const fakeChess = await smock.fake(chessAbi, {address: chess.address})
fakeChess.balanceOf.whenCalledWith(keeper._address).returns(9)
fakeChess.balanceOf.whenCalledWith(officialSudoPool.address).returns(100)

fakeChess.transferFrom.returns(true)

const fakeKeep3r = await smock.fake(keep3rAbi, {address: KEEP3R})
fakeKeep3r.isKeeper.returns(true)
fakeKeep3r.worked.returns

In [ ]:
const NEW_BOARD = '0x03256230011111100000000000000000099999900bcdecb000000001'
const WHITE_CAP = '0x03256230011111100000000000000000099909900bcdecb000000001'
const BLACK_CAP = '0x03256230011011100000000000000000099999900bcdecb000000001'
const BOTH_CAPS = '0x03256230011011100000000000000000099909900bcdecb000000001'

In [ ]:
fakeChess.board.reset();
fakeChess.board.returnsAtCall(0, NEW_BOARD)
fakeChess.board.returnsAtCall(1, BLACK_CAP)
await evm.advanceTimeAndBlock(5*86400)
await game.connect(keeper).executeMove()

In [ ]:
const ZERO = '0x0000000000000000000000000000000000000000'
await wallet.fund(fakeChess.address, bn.toUnit(1))

await game.connect(fakeChess.wallet).onERC721Received(ZERO, ZERO, 1, ZERO)
await officialSudoPool.connect(fakeChess.wallet).onERC721Received(ZERO, ZERO, 1, ZERO)
await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

## Token URIs

In [ ]:
fakeChess.board.returns('0x0335423001101110000006009000c000099b09e00bc9ecb000000001')

const scoreboard = await game.tokenURI(0)

In [ ]:
JSON.parse(JSON.stringify(scoreboard))

In [ ]:
JSON.parse(JSON.stringify(await game.tokenURI(1)))

In [ ]:
fakeButtPlug.owner.returns(fakeButtPlug.address)
await game.mintButtPlugBadge(fakeButtPlug.address)

In [ ]:
const buttPlugBadge = await game['getBadgeId(address)'](fakeButtPlug.address)
fakeButtPlug.readMove.returns(2731)
JSON.parse(await game.tokenURI(buttPlugBadge))

## Prize claiming

In [ ]:
const tx = await game.unbondLiquidity()

await contracts.logTx(tx)

In [ ]:
const tx = await game['claimPrize(uint256)'](5)
// const tx = await game['claimPrize(uint256[])']([0,1,2,3,4])

await contracts.logTx(tx)

### Prize ceremony

In [ ]:
await evm.advanceTimeAndBlock(14*86400)
const tx = await game.withdrawLiquidity()

await contracts.logTx(tx)

In [ ]:
const tx = await game.withdrawPrize()

await contracts.logTx(tx)